In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

last_path = [[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,1],[1,0,1],[1,1,1]]
prefix = [f"{i:03d}" for i in range(505, 513)]
prefix_to_label = dict(zip(prefix, last_path))

def process_dataset(root_folder):
    image_paths = []
    label_data = []

    for roots, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 파일 이름 분석을 위해 숫자만 추출
                prefix = file[0:3]
                
                # 접두사에 따른 레이블 할당
                label = prefix_to_label.get(prefix)
                
                # 유효한 레이블이 있는 경우에만 리스트에 추가
                if label is not None:
                    image_paths.append(os.path.join(roots, file))
                    label_data.append(label)
    
    return image_paths, label_data

# 각각의 데이터셋에 대해 함수를 호출
train_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
valid_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

train_image_paths, train_label_data = process_dataset(train_folder)
valid_image_paths, valid_label_data = process_dataset(valid_folder)
test_image_paths, test_label_data = process_dataset(test_folder)

# 필요에 따라 결과를 확인하거나 다른 처리를 수행
print(len(train_image_paths), len(train_label_data))
print(len(valid_image_paths), len(valid_label_data))
print(len(test_image_paths), len(test_label_data))

7874 7874
1691 1691
1687 1687


In [2]:
def resize_img(image_paths):
    images_resized = []  # 리사이즈된 이미지를 저장할 리스트
    for image_path in image_paths:
        image = cv2.imread(image_path)  # 각 이미지 경로로부터 이미지를 읽음
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 색상 변환
        image_resized = cv2.resize(image, (128, 128))  # 이미지 리사이즈
        images_resized.append(image_resized)  # 결과 리스트에 추가
    images_resized = np.array(images_resized) / 255.0  # numpy 배열로 변환 및 정규화
    return images_resized

train_image_resized = resize_img(train_image_paths)
valid_image_resized = resize_img(valid_image_paths)
test_image_resized = resize_img(test_image_paths)

train_label_data = np.array(train_label_data)
valid_label_data = np.array(valid_label_data)
test_label_data = np.array(test_label_data)

print('done')

print(type(valid_label_data))  # 데이터 타입 확인
if isinstance(valid_label_data, np.ndarray):
    print(valid_label_data.shape)  # numpy 배열인 경우, 모양 확인
print(type(train_label_data))  # 데이터 타입 확인
if isinstance(train_label_data, np.ndarray):
    print(train_label_data.shape)  # numpy 배열인 경우, 모양 확인


done
<class 'list'>
<class 'list'>


In [3]:
from keras.optimizers import Adam
# 모델 구성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.5),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.5),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    # 멀티라벨 분류를 위한 sigmoid 활성화 함수 사용
    Dense(3, activation='sigmoid')
])


# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])



earlystopping = EarlyStopping(monitor='val_loss', patience=7, mode='min', verbose=1)


# 이제 모델 학습을 실행할 수 있습니다.
model.fit(train_image_resized, 
          train_label_data, 
          validation_data=(valid_image_resized, valid_label_data), 
          epochs=25, 
          batch_size=32,
         callbacks=[earlystopping])

# 테스트 데이터셋으로 모델 평가
test_loss, test_acc = model.evaluate(test_image_resized, test_label_data)
print('\n테스트 정확도:', test_acc)


Epoch 1/25
247/247 [==============================] - 7s 17ms/step - loss: 0.6926 - accuracy: 0.3820 - val_loss: 0.6841 - val_accuracy: 0.2779
Epoch 2/25
247/247 [==============================] - 4s 15ms/step - loss: 0.6840 - accuracy: 0.2951 - val_loss: 0.6676 - val_accuracy: 0.2839
Epoch 3/25
247/247 [==============================] - 4s 15ms/step - loss: 0.6613 - accuracy: 0.3373 - val_loss: 0.6450 - val_accuracy: 0.3335
Epoch 4/25
247/247 [==============================] - 4s 15ms/step - loss: 0.6396 - accuracy: 0.3655 - val_loss: 0.6257 - val_accuracy: 0.2975
Epoch 5/25
247/247 [==============================] - 4s 15ms/step - loss: 0.6142 - accuracy: 0.3642 - val_loss: 0.5978 - val_accuracy: 0.3649
Epoch 6/25
247/247 [==============================] - 4s 15ms/step - loss: 0.5892 - accuracy: 0.4103 - val_loss: 0.5785 - val_accuracy: 0.3708
Epoch 7/25
247/247 [==============================] - 4s 15ms/step - loss: 0.5640 - accuracy: 0.4261 - val_loss: 0.5607 - val_accuracy: 0.4352